In [1]:
import time
from threading import Thread
from math import inf

In [2]:
def compare(word, solution):
    response = 0
    for w in range(len(word)):
        response = response * 10
        if word[w] == solution[w]:
            response = response + 2
        elif word[w] in solution:
            response = response + 1
    return response

def get_sets(word, remaining):
    combs = {}
    for solution in remaining:
        score = compare(word, solution)
        combs.setdefault(score, [])
        combs.setdefault(score, []).append(solution)
    return combs

def filter_set(remaining, word, response):
    return [r for r in remaining if compare(word, r) == response]

In [3]:
# Each node need to know: remaining feasible set
# Optionally: parent, and current move counter
class Node:
    def __init__(self, parent, remaining, counter, info):
        self.parent = parent
        self.children = {}
        self.child_scores = {}
        self.remaining = remaining
        self.counter = counter
        self.expanded = False
        self.finished = False
        self.pruned = False
        self.avg_moves = None
        self.best_move = ''
        self.local_bound = inf
        self.info = info
        # print(f"New Node: {parent} {counter} {info} {remaining}")

    def get_status(self):
        return {
            'id': id(self),
            'parent': self.parent,
            'remaining': self.remaining,
            'counter': self.counter,
            'expanded': self.expanded,
            'finished': self.finished,
            'pruned': self.pruned,
            'avg_moves': self.avg_moves,
            'children': len(self.children),
            'best_move': self.best_move,
            'info': self.info
        }

    def check(self, global_bound):
        score = {}
        completed = True
        if len(self.children) > 0:
            set_size = len(self.remaining)
            for word, word_children in self.children.items():
                if self.child_scores.get(word) is not None:
                    continue
                word_done = True
                # print(word, word_children)
                for response, response_children in word_children.items():
                    all_done = all([i.finished for i in response_children])
                    if not all_done:
                        completed = False
                        word_done = False
                        continue
                    values = [i.avg_moves for i in response_children]
                    # print(response_children[0].get_status())
                    # if all_done:
                    #     print("ALL DONE: ")
                    #     print(self.get_status())

                    best_case = min(values)
                    response_score = len(response_children)/set_size * best_case
                    score[word] = score.get(word, 0) + response_score
                if word_done:
                    print(f"New value added: {self.info} {word} {score[word]}")
                    self.child_scores[word] = score[word]
                    self.local_bound = min(self.local_bound, score[word])

        delete_words = []
        for word in score:
            if score[word] > global_bound or score[word] > self.local_bound: # already worse than best case
                for response_children in self.children[word].values():
                    for child in response_children:
                        child.prune()
                delete_words.append(word)
                del self.children[word]
        # for word in delete_words:
        #     del score[word]

        if len(self.children) == 0:
            self.prune()

        if completed:
            self.finished = True
            best_word = min(score, key=score.get)
            self.avg_moves = score[best_word]
            self.best_move = best_word
            if self.counter <= 2:
                print("COMPLETED")
                print(self.get_status())
                print(score)
            if self.parent:
                self.parent.check(min(self.local_bound, global_bound))

    def prune(self):
        print(f"PRUNED {self.info}")
        self.pruned = False
        for word, word_children in self.children.items():
            for response, response_children in word_children.items():
                for child in response_children:
                    if not child.finished:
                        child.prune()

# Each worker picks up next available job possible and expands the set
class Worker:
    def __init__(self, id, controller):
        self.id = id
        self.controller = controller
    
    def __call__(self):
        print(f"Start worker {self.id}")
        while self.controller.has_more():
            self.run()
        print(f"End worker: {self.id}")

    def run(self):
        node = self.controller.get_next()
        if node is None:
            time.sleep(1)
            return

        if node.pruned:
            node.finished = True
            node.avg_moves = None
            return

        if len(node.remaining) == 1:
            node.finished = True
            node.avg_moves = node.counter
            # print(node.get_status())
            parent = node.parent
            if parent:
                parent.check(self.controller.get_bound())
            return

        search_space = self.controller.get_search_space(node)
        for word in search_space:
            if word in node.info:
                continue
            sets = get_sets(word, node.remaining)
            children_dict = node.children.setdefault(word, dict())
            # print(word, sets)
            for response in sets:
                child = Node(node, sets[response].copy(), node.counter + 1, node.info.copy() + [word, response])
                children_dict.setdefault(response, []).append(child)
                self.controller.add_node(child)
            
            node.expanded = True


class Controller():

    def __init__(self, first_node):
        self.active_nodes = [first_node]
        self.first_node = first_node
        self.global_bound = inf
        # self.workers = [Worker(1, self)]

    def start(self):
        w = Worker(1, self)
        # self.threads = Thread(target=w, name="thread")
        # self.threads.start()
        # self.threads.join()
        w.__call__()

    def get_search_space(self, node):
        return node.remaining

    def add_node(self, node):
        self.active_nodes.append(node)

    def get_next(self, method='smallest_set'):
        if len(self.active_nodes) > 0:
            # ordered
            if method == 'ordered':
                node = self.active_nodes.pop(0)
            elif method == 'smallest_set':
                set_sizes = [[-i.counter, len(i.remaining)] for i in self.active_nodes]
                min_index = set_sizes.index(min(set_sizes))
                node = self.active_nodes.pop(min_index)
            print(f"Processing {id(node)}")
            return node
        else:
            return None

    def get_bound(self):
        return self.global_bound

    def has_more(self):
        print(f"Asking for more: {len(self.active_nodes)}")
        return len(self.active_nodes) > 0 or not self.first_node.finished


In [4]:
with open("solutions.txt") as f:
    text = f.read()
    solution_set = [i.replace('"', '').replace(' ', '') for i in text.split(",")]
solution_set[0:5]

['cigar', 'rebut', 'sissy', 'humph', 'awake']

In [5]:
target = solution_set[0:3]

initial_node = Node(None, target, 1, ['StartNode'])
c = Controller(initial_node)
c.start()
initial_node.get_status()

Start worker 1
Asking for more: 1
Processing 2751342239808
Asking for more: 9
Processing 2751342222880
Asking for more: 8
Processing 2751342221536
Asking for more: 7
Processing 2751342222400
New value added: ['StartNode'] cigar 2.0
Asking for more: 6
Processing 2751342221680
Asking for more: 5
Processing 2751342221584
Asking for more: 4
Processing 2751342222928
New value added: ['StartNode'] rebut 2.0
Asking for more: 3
Processing 2751342222784
Asking for more: 2
Processing 2751342221872
Asking for more: 1
Processing 2751342221632
New value added: ['StartNode'] sissy 2.0
COMPLETED
{'id': 2751342239808, 'parent': None, 'remaining': ['cigar', 'rebut', 'sissy'], 'counter': 1, 'expanded': True, 'finished': True, 'pruned': False, 'avg_moves': 2.0, 'children': 3, 'best_move': 'sissy', 'info': ['StartNode']}
{'sissy': 2.0}
Asking for more: 0
End worker: 1


{'id': 2751342239808,
 'parent': None,
 'remaining': ['cigar', 'rebut', 'sissy'],
 'counter': 1,
 'expanded': True,
 'finished': True,
 'pruned': False,
 'avg_moves': 2.0,
 'children': 3,
 'best_move': 'sissy',
 'info': ['StartNode']}